In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#importing lib
import os
import seaborn as sb
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder


In [ ]:
path="/content/drive/MyDrive/16.0/training"
train=image_dataset_from_directory(path,batch_size=32,image_size=(224, 224), shuffle=True)

In [ ]:
path="/content/drive/MyDrive/16.0/validation"
valid=image_dataset_from_directory(path,batch_size=32,image_size=(224, 224), shuffle=False)

In [ ]:
path="/content/drive/MyDrive/16.0/testing"
test=image_dataset_from_directory(path,batch_size=32,image_size=(224, 224), shuffle=False)

Create a list to store spectograms and labels

In [ ]:
spectrograms = []
labels = []

Label encoding process

In [ ]:
spectrogram_folder= "/content/drive/MyDrive/16.0_noisy/testing"

In [ ]:
input_shape = (224,224, 3)

In [ ]:
from tqdm import tqdm

In [ ]:
for file_name in tqdm(os.listdir(spectrogram_folder)):
        temp=os.path.join(spectrogram_folder,file_name)
        for i in os.listdir(temp):
          # Load the spectrogram image and convert it to a numpy array

          img_path = os.path.join(spectrogram_folder, file_name,i)
          img = load_img(img_path, target_size=(224,224))
          img_array = img_to_array(img)

          # Append the spectrogram image and its corresponding label to the lists
          spectrograms.append(img_array)
          label =file_name
          labels.append(label)
          # print(label)

In [ ]:
spectrograms = np.array(spectrograms)
labels = np.array(labels)


In [ ]:
import pandas as pd

In [ ]:
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
df = pd.DataFrame({'Original Label': labels, 'Encoded Label': encoded_labels})
print(df)

In [ ]:
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

# Print mapping
for original, encoded in label_mapping.items():
    print(f"'{original}' → {encoded}")

In [ ]:
# Convert the labels to one-hot encoding
labels = to_categorical(label_encoder.transform(labels)) # Change to this line
print(labels)

In [ ]:
from sklearn.model_selection import train_test_split
train_spectrograms, test_spectrograms, train_labels, test_labels = train_test_split(spectrograms, labels, test_size=0.2, stratify=labels)

In [ ]:
train_spectrograms.shape

Improved-InceptionV3

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

from tensorflow.keras.optimizers import Adam

# Parameters

num_classes = 10  # Change this to the number of classes in your dataset
learning_rate = 0.0001


# Load the Xception model pre-trained on ImageNet
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Combine the base model with the custom layers
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model to retain pre-trained weights
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train, validation_data=valid, epochs=10, batch_size=32)
# # Print model summary
model.summary()






#  fine-tune the entire model by unfreezing the base model layers
for layer in base_model.layers:
    layer.trainable = True

# Re-compile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=learning_rate / 10),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit(train, validation_data=valid, epochs=20, batch_size=32)


Improved-Xception

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

from tensorflow.keras.optimizers import Adam

# Parameters

num_classes = 10  # Change this to the number of classes in your dataset
learning_rate = 0.0001
batch_size = 32
epochs = 10

# Load the Xception model pre-trained on ImageNet
base_model = Xception(weights='imagenet', include_top=False, input_shape=input_shape)

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Combine the base model with the custom layers
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model to retain pre-trained weights
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train, validation_data=valid, epochs=10, batch_size=32)
# Print model summary
model.summary()






#  fine-tune the entire model by unfreezing the base model layers
for layer in base_model.layers:
    layer.trainable = True

# Re-compile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=learning_rate / 10),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit(train, validation_data=valid, epochs=20, batch_size=32)

Improved-ResNet152V2

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

from tensorflow.keras.optimizers import Adam

# Parameters

num_classes = 10  # Change this to the number of classes in your dataset
learning_rate = 0.0001


# Load the Xception model pre-trained on ImageNet
base_model = ResNet152V2(weights='imagenet', include_top=False, input_shape=input_shape)

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Combine the base model with the custom layers
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model to retain pre-trained weights
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train, validation_data=valid, epochs=10, batch_size=32)
# # Print model summary
# model.summary()






#  fine-tune the entire model by unfreezing the base model layers
for layer in base_model.layers:
    layer.trainable = True

# Re-compile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=learning_rate / 10),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
history=model.fit(train, validation_data=valid, epochs=20, batch_size=32)

Time

In [ ]:
import time
import numpy as np

# Sample input from validation set
sample_input = next(iter(valid))[0][:1]  # Take 1 image from the validation set

# Warm-up (to avoid first-run overhead)
_ = model.predict(sample_input)

# Measure inference time
start_time = time.time()
_ = model.predict(sample_input)
end_time = time.time()

inference_time = end_time - start_time
print(f"Inference time per image: {inference_time:.4f} seconds")


In [ ]:
model.save("model_3.h5")

import os
model_size = os.path.getsize("model_3.h5") / (1024 ** 2)  # Convert to MB
print(f"Model size: {model_size:.2f} MB")


In [ ]:
trainable_params = np.sum([np.prod(v.get_shape()) for v in model.trainable_weights])
non_trainable_params = np.sum([np.prod(v.get_shape()) for v in model.non_trainable_weights])
total_params = trainable_params + non_trainable_params

print(f"Trainable parameters: {trainable_params:,}")
print(f"Non-trainable parameters: {non_trainable_params:,}")
print(f"Total parameters: {total_params:,}")


In [ ]:
# # Compile the model
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# # Train the model on the training data
# train_history = model.fit(train_spectrograms, train_labels, epochs=40, batch_size=8, validation_split=0.3)

# # Evaluate the model on the testing data
# test_loss, test_accuracy = model.evaluate(test_spectrograms, test_labels)

# # Optionally, you can also access the training history
train_loss = history.history['loss']
train_accuracy = history.history['accuracy']
val_loss = history.history['val_loss']
val_accuracy = history.history['val_accuracy']

In [ ]:
# # Plot training and validation loss
# plt.plot(train_loss, label='Training Loss')
# plt.plot(val_loss, label='Validation Loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.title("SNR 12")
# plt.legend()
# plt.show()

# Plot training and validation accuracy
plt.plot(train_accuracy, label='Training Accuracy')
plt.plot(val_accuracy, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title("SNR 12")
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# Generate predictions for the test set
y_pred = model.predict(test_spectrograms)
y_pred_classes = np.argmax(y_pred, axis=1)

# Compute the confusion matrix
conf_matrix = confusion_matrix(np.argmax(test_labels, axis=1), y_pred_classes)

# Define the class labels
class_labels = label_encoder.classes_

# Print the confusion matrix with class labels
print("Confusion matrix:")
print(conf_matrix)
print("")

# Generate and print the classification report
class_report = classification_report(np.argmax(test_labels, axis=1), y_pred_classes, target_names=class_labels)
print("Classification report:")
print(class_report)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Greens", xticklabels=class_labels, yticklabels=class_labels)

# Add labels and title
plt.xlabel("Predicted labels")
plt.ylabel("True labels")
plt.title("Confusion Matrix for SNR 12.0")

# Display the plot
plt.show()

Visulaizing CNN Architecture

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]

    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]

    heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_outputs), axis=-1)
    heatmap = np.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()


In [ ]:
def superimpose_heatmap(img, heatmap, alpha=0.4):
    heatmap_resized = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap_colored = cv2.applyColorMap(np.uint8(255 * heatmap_resized), cv2.COLORMAP_JET)
    superimposed_img = heatmap_colored * alpha + img
    return np.uint8(superimposed_img)


Noise image testing

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2

# -------- Model and Configuration --------
img_path = '/content/drive/MyDrive/16.0/testing/OFDM_QAM64/replica_5.png'
input_size = (224, 224)
last_conv_layer_name = 'post_relu'  # Use the last conv layer in your model

# -------- Add Colored Gaussian Noise --------
def add_colored_gaussian_noise(image_path, mean=0, std=25):
    image = cv2.imread(image_path)
    image = cv2.resize(image, input_size)
    image = image.astype(np.float32)
    noise = np.random.normal(loc=mean, scale=std, size=image.shape).astype(np.float32)
    noisy_image = np.clip(image + noise, 0, 255).astype(np.uint8)
    return noisy_image

# -------- Grad-CAM function --------
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model(
        inputs=[model.inputs],
        outputs=[model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]

    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

# -------- Superimpose heatmap --------
def superimpose_heatmap(original_img, heatmap, alpha=0.4, colormap=cv2.COLORMAP_JET):
    heatmap = cv2.resize(heatmap, (original_img.shape[1], original_img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap_colored = cv2.applyColorMap(heatmap, colormap)
    original_bgr = cv2.cvtColor(original_img, cv2.COLOR_RGB2BGR)
    superimposed_img = cv2.addWeighted(heatmap_colored, alpha, original_bgr, 1 - alpha, 0)
    return superimposed_img

# -------- Generate noisy image and preprocess --------
noisy_img = add_colored_gaussian_noise(img_path)
noisy_img_rgb = cv2.cvtColor(noisy_img, cv2.COLOR_BGR2RGB)
input_noisy = np.expand_dims(noisy_img_rgb, axis=0) / 255.0  # Normalize

# -------- Predict and Grad-CAM --------
pred_probs = model.predict(input_noisy)[0]
predicted_class = np.argmax(pred_probs)

heatmap_noisy = make_gradcam_heatmap(input_noisy, model, last_conv_layer_name, pred_index=predicted_class)
cam_image_noisy = superimpose_heatmap(noisy_img_rgb, heatmap_noisy)

# -------- Display Results --------
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
original = cv2.imread(img_path)
original = cv2.resize(original, input_size)
plt.title("Original Image")
plt.imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
plt.axis('off')

plt.subplot(1, 3, 2)
plt.title("Colored Noisy Image")
plt.imshow(noisy_img_rgb)
plt.axis('off')

plt.subplot(1, 3, 3)
plt.title("Grad-CAM (Noisy Image)")
plt.imshow(cv2.cvtColor(cam_image_noisy, cv2.COLOR_BGR2RGB))
plt.axis('off')

plt.tight_layout()
plt.show()
